In [2]:
#ensembling
from mlxtend.classifier import StackingCVClassifier
import numpy as np
import pandas as pd
import sys, os, glob
import imp
import seaborn as sns
sys.path.append('./src/')
import common_utils,my_ensembler, feature_utils, defines, model_utils, my_bert
from sklearn.neighbors import KNeighborsClassifier
from sklearn_crfsuite import scorers, CRF
from sklearn.pipeline import Pipeline
import json
from operator import itemgetter
from sklearn_crfsuite.utils import flatten
from sklearn_crfsuite.metrics import sequence_accuracy_score
from sklearn.linear_model import LogisticRegression
import scipy
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import classes
import pickle
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

In [3]:
dir_name='reparse'
seq_len=15
step=15
cv_split_name='cv_splits_10'
merged_str='scaled.binary0.50_merged'
doc_indices=np.arange(1,81)
doc_indices
test_doc_num=8
seed=42

In [132]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### Fix double missing + double stand-alone

In [4]:
all_pred_crf01=common_utils.load_db(dir_name,"bert.crf.svc.new.76.label.01")

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/bert.crf.svc.new.76.label.01.csv,  index False


In [5]:
my_tables=classes.MyScoreSummarizer(all_pred_crf01)
my_tables.get_all_scores()

MyScoreSummarizer init called
MyPredFixer init called
WindowDiff init called
bert
near misses to be fixed
 {'tot': 346, 'fp': 162, 'fn': 184}
1039 middle misses to be fixed
1428 stande alone to be fixed
crf
near misses to be fixed
 {'tot': 100, 'fp': 48, 'fn': 52}
23 middle misses to be fixed
24 stande alone to be fixed
ens
near misses to be fixed
 {'tot': 156, 'fp': 57, 'fn': 99}
281 middle misses to be fixed
220 stande alone to be fixed
MyReport init called
MyReport init called
MyReport init called


In [47]:
del all_pred_crf01
del my_tables

### Separate lexical  & non-lexical features

In [220]:
crf_cv_features=common_utils.load_db(dir_name,"crf.features.new.76")

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/crf.features.new.76.csv,  index False


In [228]:
crf_cv_features.shape[0]

11948

In [221]:
# del crf_cv_features

In [226]:
tf_f_num=crf_cv_features[crf_cv_features['attr'].str.contains('tfidf')].shape[0]
print(tf_f_num,tf_f_num/crf_cv_features.shape[0])

11138 0.9322062269835956


In [231]:
neighbor_f_num=crf_cv_features[crf_cv_features['attr'].str.contains("\+|\-")].shape[0]
print(neighbor_f_num,neighbor_f_num*100/(crf_cv_features.shape[0]-tf_f_num))

698 86.17283950617283


In [76]:
nar_pos=crf_cv_features[(crf_cv_features['label']=='is_nar')&(crf_cv_features['mean']>0)][['mean','attr','string']].sort_values(by='mean',ascending=False).reset_index(drop=True).copy()
nar_neg=crf_cv_features[(crf_cv_features['label']=='is_nar')&(crf_cv_features['mean']<0)][['mean','attr','string']].sort_values(by='mean',ascending=True).reset_index(drop=True).copy()

In [77]:
nar_neg

,mean,attr,string
0,-1.076382,tfidf_char_wb_8866,אות
1,-1.066789,tfidf_char_wb_7336,ש
2,-1.039545,tfidf_lemma_1539,כלל
3,-1.006314,tfidf_word_2521,ישיבה
4,-0.993223,tfidf_word_812,בשביל
...,...,...,...
1844,-0.000001,tfidf_word_2459,יכולים
1845,-0.000001,tfidf_char_wb_24856,משנה
1846,-0.000001,tfidf_char_wb_30644,שונ
1847,-0.000001,tfidf_char_wb_11500,גור


### Meta-features

In [234]:
crf_cv_features[(~crf_cv_features['attr'].str.contains('tfidf'))&(crf_cv_features['label']=='is_nar')]['mean'].describe()

count   400.00000
mean     -0.01439
std       0.09502
min      -0.81264
25%      -0.01949
50%      -0.00106
75%       0.00860
max       0.44987
Name: mean, dtype: float64

In [235]:
crf_cv_features[(crf_cv_features['attr'].str.contains('tfidf'))&(crf_cv_features['label']=='is_nar')]['mean'].describe()

count   4180.00000
mean       0.07072
std        0.29742
min       -1.07638
25%       -0.00181
50%        0.00083
75%        0.03376
max        3.95124
Name: mean, dtype: float64

In [166]:
nar_pos_meta=nar_pos[~nar_pos['attr'].str.contains('tfidf')].copy()
nar_neg_meta=nar_neg[~nar_neg['attr'].str.contains('tfidf')].copy()

In [157]:
pos_threshold=np.percentile(nar_pos_meta['mean'],95)
pos_threshold

0.10629587999999998

In [158]:
neg_threshold=np.percentile(nar_neg_meta['mean'],5)
neg_threshold

-0.19407376999999998

In [171]:
nar_pos_meta=nar_pos_meta[nar_pos_meta['mean']>=pos_threshold]

In [172]:
nar_pos_meta

,mean,attr,string
224,0.44987,+1:f_per_2,NaN
288,0.35812,-1:POSTAG_INTJ,NaN
328,0.30548,f_tense_PAST,NaN
392,0.24916,sent_len,NaN
432,0.21906,is_client,NaN
536,0.14962,+1:is_question,NaN
544,0.14692,TOKEN,NaN
632,0.11171,-2:POSTAG_NNP,NaN
644,0.10832,-1:is_client,NaN


In [237]:
nar_pos_meta.shape[0],nar_neg_meta.shape[0]

(9, 12)

In [177]:
nar_neg_meta=nar_neg_meta[nar_neg_meta['mean']<neg_threshold]
nar_neg_meta.sort_values(by='mean',inplace=True,ascending=False)
nar_neg_meta

,mean,attr,string
112,-0.19445,is_question,NaN
109,-0.19961,+3:POSTAG_DTT,NaN
95,-0.22185,+2:f_tense_IMPERATIVE,NaN
70,-0.27827,-1:f_suf_per_2.0,NaN
58,-0.30900,+1:POSTAG_COP,NaN
56,-0.31393,+3:POSTAG_CD,NaN
42,-0.36185,f_per_2,NaN
36,-0.39718,+1:POSTAG_CDT,NaN
24,-0.48225,+3:f_tense_IMPERATIVE,NaN
14,-0.61748,+1:f_suf_num_P,NaN


In [257]:
nar_meta=pd.concat([nar_pos_meta,nar_neg_meta],axis=0).drop('string',axis=1) #,ignore_index=True)
nar_meta.reset_index(inplace=True)
nar_meta.rename({'index':'rank'},axis=1,inplace=True)
nar_meta

,rank,mean,attr
0,224,0.44987,+1:f_per_2
1,288,0.35812,-1:POSTAG_INTJ
2,328,0.30548,f_tense_PAST
3,392,0.24916,sent_len
4,432,0.21906,is_client
5,536,0.14962,+1:is_question
6,544,0.14692,TOKEN
7,632,0.11171,-2:POSTAG_NNP
8,644,0.10832,-1:is_client
9,112,-0.19445,is_question


In [259]:
nar_meta['prefix']=nar_meta['attr'].astype(str).str[:3]
nar_meta

,rank,mean,attr,prefix
0,224,0.44987,+1:f_per_2,+1:
1,288,0.35812,-1:POSTAG_INTJ,-1:
2,328,0.30548,f_tense_PAST,f_t
3,392,0.24916,sent_len,sen
4,432,0.21906,is_client,is_
5,536,0.14962,+1:is_question,+1:
6,544,0.14692,TOKEN,TOK
7,632,0.11171,-2:POSTAG_NNP,-2:
8,644,0.10832,-1:is_client,-1:
9,112,-0.19445,is_question,is_


In [264]:
nar_meta['prefix'].str.contains("\-1",na=False)

0     False
1      True
2     False
3     False
4     False
5     False
6     False
7     False
8      True
9     False
10    False
11    False
12     True
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
Name: prefix, dtype: bool

In [269]:
nar_meta['prefix']=nar_meta['attr'].astype(str).str[:3]
nar_meta.loc[nar_meta['prefix'].str.contains(r"\-3",na=False),'order']=-3
nar_meta.loc[nar_meta['prefix'].str.contains(r"\-2",na=False),'order']=1
nar_meta.loc[nar_meta['prefix'].str.contains(r"\-1",na=False),'order']=2
nar_meta.loc[~nar_meta['prefix'].str.contains(r"\+|\-",na=False),'order']=3
nar_meta.loc[nar_meta['prefix'].str.contains(r"\+1",na=False),'order']=4
nar_meta.loc[nar_meta['prefix'].str.contains(r"\+2",na=False),'order']=5
nar_meta.loc[nar_meta['prefix'].str.contains(r"\+3",na=False),'order']=6
nar_meta

,rank,mean,attr,prefix,order
0,224,0.44987,+1:f_per_2,+1:,4.00000
1,288,0.35812,-1:POSTAG_INTJ,-1:,2.00000
2,328,0.30548,f_tense_PAST,f_t,3.00000
3,392,0.24916,sent_len,sen,3.00000
4,432,0.21906,is_client,is_,3.00000
5,536,0.14962,+1:is_question,+1:,4.00000
6,544,0.14692,TOKEN,TOK,3.00000
7,632,0.11171,-2:POSTAG_NNP,-2:,1.00000
8,644,0.10832,-1:is_client,-1:,2.00000
9,112,-0.19445,is_question,is_,3.00000


In [272]:
nar_meta.sort_values(by=['order','mean'],ascending=False)

,rank,mean,attr,prefix,order
10,109,-0.19961,+3:POSTAG_DTT,+3:,6.00000
14,56,-0.31393,+3:POSTAG_CD,+3:,6.00000
17,24,-0.48225,+3:f_tense_IMPERATIVE,+3:,6.00000
20,7,-0.81264,+3:POSTAG_CDT,+3:,6.00000
11,95,-0.22185,+2:f_tense_IMPERATIVE,+2:,5.00000
0,224,0.44987,+1:f_per_2,+1:,4.00000
5,536,0.14962,+1:is_question,+1:,4.00000
13,58,-0.30900,+1:POSTAG_COP,+1:,4.00000
16,36,-0.39718,+1:POSTAG_CDT,+1:,4.00000
18,14,-0.61748,+1:f_suf_num_P,+1:,4.00000


In [275]:
nar_meta_name=common_utils.load_db(dir_name,"nar_meta_name")

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/nar_meta_name.csv,  index False


In [276]:
nar_meta_name

,idx,rank,mean,attr,name
0,0,224,0.44987,+1:f_per_2,+1 second person
1,1,288,0.35812,-1:POSTAG_INTJ,-1 interjection
2,2,328,0.30548,f_tense_PAST,past tense
3,3,392,0.24916,sent_len,len (char)
4,4,432,0.21906,is_client,is client
5,5,536,0.14962,+1:is_question,+1 is question
6,6,544,0.14692,TOKEN,len (words)
7,7,632,0.11171,-2:POSTAG_NNP,-2 proper noun
8,8,644,0.10832,-1:is_client,-1 is client
9,9,112,-0.19445,is_question,is question


In [285]:
merged=nar_meta.merge(nar_meta_name[['attr','name']],left_on='attr',right_on='attr',validate='one_to_one')
merged.sort_values(by=['order','mean'],inplace=True)
merged

,index,rank,mean,attr,prefix,order,name
0,19,11,-0.70300,-2:f_suf_per_2.0,-2:,1.00000,-2 second person
1,7,632,0.11171,-2:POSTAG_NNP,-2:,1.00000,-2 proper noun
2,12,70,-0.27827,-1:f_suf_per_2.0,-1:,2.00000,-1 second person suffix
3,8,644,0.10832,-1:is_client,-1:,2.00000,-1 is client
4,1,288,0.35812,-1:POSTAG_INTJ,-1:,2.00000,-1 interjection
5,15,42,-0.36185,f_per_2,f_p,3.00000,second person
6,9,112,-0.19445,is_question,is_,3.00000,is question
7,6,544,0.14692,TOKEN,TOK,3.00000,len (words)
8,4,432,0.21906,is_client,is_,3.00000,is client
9,3,392,0.24916,sent_len,sen,3.00000,len (char)


In [179]:
nar_meta.index.tolist()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [286]:
common_utils.save_db(merged,dir_name,"nar_meta_order",True)

Saving /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/nar_meta_order.csv, 
index True
float_format %.5f


In [184]:
nar_meta.to_csv(sep=' ')

' index mean attr\n0 224 0.4498708 +1:f_per_2\n1 288 0.3581204 -1:POSTAG_INTJ\n2 328 0.3054827 f_tense_PAST\n3 392 0.2491604 sent_len\n4 432 0.2190585999999999 is_client\n5 536 0.1496198 +1:is_question\n6 544 0.1469164999999999 TOKEN\n7 632 0.1117094999999999 -2:POSTAG_NNP\n8 644 0.1083206 -1:is_client\n9 112 -0.1944495 is_question\n10 109 -0.1996079999999999 +3:POSTAG_DTT\n11 95 -0.2218461 +2:f_tense_IMPERATIVE\n12 70 -0.27827 -1:f_suf_per_2.0\n13 58 -0.3089964 +1:POSTAG_COP\n14 56 -0.3139308 +3:POSTAG_CD\n15 42 -0.3618482 f_per_2\n16 36 -0.3971829 +1:POSTAG_CDT\n17 24 -0.4822532 +3:f_tense_IMPERATIVE\n18 14 -0.6174776 +1:f_suf_num_P\n19 11 -0.7030035714285715 -2:f_suf_per_2.0\n20 7 -0.8126399 +3:POSTAG_CDT\n'

In [205]:
nn_pos=crf_cv_features[(crf_cv_features['label']=='not_nar')&(crf_cv_features['mean']>0)][['mean','attr','string']].sort_values(by='mean',ascending=False).reset_index(drop=True).copy()
nn_neg=crf_cv_features[(crf_cv_features['label']=='not_nar')&(crf_cv_features['mean']<0)][['mean','attr','string']].sort_values(by='mean',ascending=True).reset_index(drop=True).copy()

In [206]:
nn_pos_meta=nn_pos[~nn_pos['attr'].str.contains('tfidf')].copy()
nn_neg_meta=nn_neg[~nn_neg['attr'].str.contains('tfidf')].copy()

In [207]:
pos_nn_threshold=np.percentile(nn_pos_meta['mean'],95)
pos_nn_threshold

0.19275871499999989

In [208]:
neg_nn_threshold=np.percentile(nn_neg_meta['mean'],5)
neg_nn_threshold

-0.1152301999999999

In [209]:
nn_pos_meta=nn_pos_meta[nn_pos_meta['mean']>=pos_threshold]
nn_pos_meta

,mean,attr,string
179,0.93537,-2:f_suf_per_2.0,NaN
209,0.81264,+3:POSTAG_CDT,NaN
275,0.61748,+1:f_suf_num_P,NaN
357,0.48225,+3:f_tense_IMPERATIVE,NaN
428,0.39718,+1:POSTAG_CDT,NaN
459,0.36424,f_suf_per_2.0,NaN
463,0.36185,f_per_2,NaN
502,0.31393,+3:POSTAG_CD,NaN
507,0.30900,+1:POSTAG_COP,NaN
626,0.22185,+2:f_tense_IMPERATIVE,NaN


In [211]:
nn_neg_meta=nn_neg_meta[nn_neg_meta['mean']<neg_threshold]
nn_neg_meta.sort_values(by='mean',inplace=True,ascending=False)
nn_neg_meta

,mean,attr,string
506,-0.21906,is_client,NaN
460,-0.24916,sent_len,NaN
372,-0.30548,f_tense_PAST,NaN
328,-0.34596,+2:POSTAG_JJT,NaN
320,-0.35812,-1:POSTAG_INTJ,NaN
246,-0.44987,+1:f_per_2,NaN
156,-0.64022,+3:POSTAG_JJT,NaN


In [213]:
nn_meta=pd.concat([nn_pos_meta,nn_neg_meta],axis=0).drop('string',axis=1) #,ignore_index=True)
nn_meta.reset_index(inplace=True)
nn_meta.rename({'index':'rank'},axis=1,inplace=True)
nn_meta

,rank,mean,attr
0,179,0.93537,-2:f_suf_per_2.0
1,209,0.81264,+3:POSTAG_CDT
2,275,0.61748,+1:f_suf_num_P
3,357,0.48225,+3:f_tense_IMPERATIVE
4,428,0.39718,+1:POSTAG_CDT
5,459,0.36424,f_suf_per_2.0
6,463,0.36185,f_per_2
7,502,0.31393,+3:POSTAG_CD
8,507,0.30900,+1:POSTAG_COP
9,626,0.22185,+2:f_tense_IMPERATIVE


In [289]:
nn_meta_name=common_utils.load_db(dir_name,"nn_meta_name")

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/nn_meta_name.csv,  index False


In [287]:
imp.reload(common_utils)
nn_meta_order=common_utils.order_meta_features(nn_meta)
nn_meta_order

,rank,mean,attr,prefix,order
1,209,0.81264,+3:POSTAG_CDT,+3:,6.00000
3,357,0.48225,+3:f_tense_IMPERATIVE,+3:,6.00000
7,502,0.31393,+3:POSTAG_CD,+3:,6.00000
10,664,0.19961,+3:POSTAG_DTT,+3:,6.00000
17,734,0.16604,+3:POSTAG_INTJ,+3:,6.00000
29,156,-0.64022,+3:POSTAG_JJT,+3:,6.00000
9,626,0.22185,+2:f_tense_IMPERATIVE,+2:,5.00000
12,681,0.19069,+2:POSTAG_CDT,+2:,5.00000
15,724,0.17002,+2:POSTAG_DTT,+2:,5.00000
26,328,-0.34596,+2:POSTAG_JJT,+2:,5.00000


In [290]:
nn_merged=nn_meta_order.merge(nn_meta_name[['attr','name']],left_on='attr',right_on='attr',validate='one_to_one')
nn_merged.sort_values(by=['order','mean'],inplace=True)
nn_merged.reset_index(inplace=True)
nn_merged

,index,rank,mean,attr,prefix,order,name
0,29,885,0.10705,-2:POSTAG_CDT,-2:,1.00000,-2 numeral delimeter
1,28,867,0.11144,-2:f_tense_BEINONI,-2:,1.00000,-2 neuter verb
2,27,179,0.93537,-2:f_suf_per_2.0,-2:,1.00000,-2 second person suffix
3,26,320,-0.35812,-1:POSTAG_INTJ,-1:,2.00000,-1 interjection
4,25,863,0.11300,-1:sent_pos_in_par,-1:,2.00000,-1 position in paragraph
5,24,754,0.15341,-1:f_tense_FUTURE,-1:,2.00000,-1 future tense
6,23,733,0.16624,-1:f_suf_gen_F,-1:,2.00000,-1 female suffix
7,22,372,-0.30548,f_tense_PAST,f_t,3.00000,past tense
8,21,460,-0.24916,sent_len,sen,3.00000,len (char)
9,20,506,-0.21906,is_client,is_,3.00000,is client


In [291]:
common_utils.save_db(nn_merged,dir_name,"nn_meta_order",True)

Saving /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/nn_meta_order.csv, 
index True
float_format %.5f


## Fix double misses

In [371]:
imp.reload(classes)
imp.reload(model_utils)
all_pred_crf01=common_utils.load_db(dir_name,"bert.crf.svc.new.76.label.01")
my_tables=classes.MyScoreSummarizer(all_pred_crf01)
my_tables.get_all_scores()

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/bert.crf.svc.new.76.label.01.csv,  index False
MyScoreSummarizer init called
MyPredFixer init called
WindowDiff init called
bert
near misses to be fixed
 {'tot': 346, 'fp': 162, 'fn': 184}
1890 middle misses to be fixed
1633 stande alone to be fixed
crf
near misses to be fixed
 {'tot': 100, 'fp': 48, 'fn': 52}
0 middle misses to be fixed
0 stande alone to be fixed
ens
near misses to be fixed
 {'tot': 156, 'fp': 57, 'fn': 99}
399 middle misses to be fixed
350 stande alone to be fixed
MyReport init called
MyReport init called
MyReport init called


In [372]:
scores, scores_dict, _, _ = model_utils.get_report_by_unit(my_tables.my_fixer.fixed_df,'ens',"split")

In [373]:
scores_dict

{0: {'0': {'precision': 0.8521947873799726,
   'recall': 0.9568733153638814,
   'f1-score': 0.9015055323780157,
   'support': 2597},
  '1': {'precision': 0.7575757575757576,
   'recall': 0.44814340588988477,
   'f1-score': 0.5631536604987932,
   'support': 781},
  'accuracy': 0.8392539964476021,
  'macro avg': {'precision': 0.804885272477865,
   'recall': 0.7025083606268832,
   'f1-score': 0.7323295964384045,
   'support': 3378},
  'weighted avg': {'precision': 0.83031868842287,
   'recall': 0.8392539964476021,
   'f1-score': 0.8232779385539563,
   'support': 3378}},
 1: {'0': {'precision': 0.8121122599704579,
   'recall': 0.9443490209549983,
   'f1-score': 0.8732528589580687,
   'support': 2911},
  '1': {'precision': 0.7162872154115587,
   'recall': 0.39138755980861245,
   'f1-score': 0.5061881188118812,
   'support': 1045},
  'accuracy': 0.7982810920121335,
  'macro avg': {'precision': 0.7641997376910084,
   'recall': 0.6678682903818054,
   'f1-score': 0.689720488884975,
   'support'

In [382]:
scores_pure_dic={}
for t in ['ens','crf','bert']:
    _, scores_pure_dic[t], _, _ = model_utils.get_report_by_unit(my_tables.my_fixer.pred_df,t,"split")

In [383]:
fixed_ens_score=pd.DataFrame()
db=pd.DataFrame()

for kk,vv in scores_pure_dic.items():
    for fold,v in vv.items():
        for l in ['0','1','weighted avg']:
            db.loc[fold,'{}_{}'.format(l,kk)]=v[l]['f1-score']

            

,0_ens,1_ens,weighted avg_ens,0_crf,1_crf,weighted avg_crf,0_bert,1_bert,weighted avg_bert
0,0.89853,0.55172,0.81835,0.88687,0.55279,0.80963,0.85753,0.52347,0.78029
1,0.87093,0.49597,0.77188,0.85659,0.46722,0.75374,0.80492,0.52371,0.73063
2,0.86791,0.51705,0.77385,0.85227,0.54064,0.76873,0.83174,0.53370,0.75184
3,0.84588,0.55607,0.76610,0.82643,0.53751,0.74690,0.78910,0.53235,0.71842
4,0.85815,0.59779,0.77550,0.84276,0.64000,0.77839,0.81768,0.54094,0.72983
5,0.87129,0.52308,0.77321,0.85557,0.51271,0.75900,0.84501,0.53488,0.75766
6,0.90666,0.54207,0.83864,0.89951,0.53502,0.83151,0.86698,0.47746,0.79431
7,0.88770,0.54881,0.81446,0.88417,0.54009,0.80980,0.79090,0.46407,0.72026
8,0.90266,0.51558,0.83403,0.89691,0.49487,0.82563,0.81908,0.42435,0.74909
9,0.82917,0.64804,0.76171,0.79018,0.62458,0.72851,0.77933,0.60267,0.71354


In [394]:
for fold,v in scores_dict.items():
    for l in ['0','1','weighted avg']:
        db.loc[fold,'{}_{}'.format(l,'fixed')]=v[l]['f1-score']

In [396]:
db
common_utils.save_db(db,dir_name,"all.f1.per_label")

Saving /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/all.f1.per_label.csv, 
index False
float_format %.5f


### Try different types of fixing

In [563]:
my_l=np.array([1,2,3])

In [564]:
my_l+=1
my_l

array([2, 3, 4])

In [557]:
my_df.index.shift(1)

NotImplementedError: This method is only implemented for DatetimeIndex, PeriodIndex and TimedeltaIndex; Got type RangeIndex

In [635]:
fix_options = {
    'sm.ssa':['single_miss','single_sa'],
    'dm.sm':['double_miss','single_miss'],
    'sm.ssa.dm':['single_miss','single_sa','double_miss'],
    'sm.ssa.dm.dsa':['single_miss','single_sa','double_miss','double_sa'],
    'tm': ['tree_miss','single_sa'],
    'tm.dm.sm': ['tree_miss','double_miss','single_miss'],
    'tm.dm.sm.ssa':  ['tree_miss','double_miss','single_miss','single_sa'],
    'tm.dm.sm.one.neigh':  ['tree_miss','double_miss','single_miss'],
    'fm.tm.dm.sm.one.neigh':  ['four_miss','tree_miss','double_miss','single_miss'],
    'fm.tm.dm.sm.ssa.dsa.one.neigh':  ['four_miss','tree_miss','double_miss','single_miss','single_sa','double_sa'],
    'fm.tm.dm.sm.ssa':  ['four_miss','tree_miss','double_miss','single_miss','single_sa'],
    'sm.2':['loop_2'],
    'sm.3':['loop_3'],
    'sm.4':['loop_4'],
    'dm.shift': ['double_miss'],
    'sm.dm.shift': ['single_miss','double_miss'],
    'sm.dm.tm.shift': ['single_miss','double_miss','tree_miss'],
    'sm.dm.tm.fm.shift': ['single_miss','double_miss','tree_miss','four_miss'],
    'sm.dm.tm.fm.ssa': ['single_miss','double_miss','tree_miss','four_miss','single_sa'],
    'sm.dm.tm.fm.ssa.dsa': ['single_miss','double_miss','tree_miss','four_miss','single_sa','double_sa'],
    'order.ssa.sm.dm.tm.fm': ['single_sa','four_miss','tree_miss','double_miss','single_miss'],
    'fm.tm.dm.sm_4n': ['four_miss','tree_miss','double_miss','single_miss','single_sa'],
    'fm.tm.dm.ssa_4n.dsa_4n': ['four_miss','tree_miss','double_miss','single_miss','double_sa','single_sa']
}

In [636]:
run_options={}
run_options={
'fm.tm.dm.ssa_4n.dsa_4n': ['four_miss','tree_miss','double_miss','single_miss','double_sa','single_sa']

}

In [637]:
imp.reload(classes)

for  name,opt in run_options.items():
    my_tables[name]=classes.MyScoreSummarizer(pred_df=all_pred_crf01,fix_list=opt)
    my_tables[name].get_all_scores()

MyScoreSummarizer init called
MyPredFixer init called
WindowDiff init called
ens
near misses to be fixed
 {'tot': 156, 'fp': 57, 'fn': 99}
332 four middle misses to be fixed
573 tree middle misses to be fixed
682 double middle misses to be fixed
773 middle misses to be fixed
138 double stande alone to be fixed
207 stande alone to be fixed
MyReport init called


In [589]:
ll=my_tables['sm.dm.tm.shift'].my_fixer.tree_middle_miss['ens']
ll

[206,
 207,
 208,
 236,
 237,
 238,
 288,
 289,
 290,
 513,
 514,
 515,
 1430,
 1431,
 1432,
 1889,
 1890,
 1891,
 2571,
 2572,
 2573,
 3228,
 3229,
 3230,
 3234,
 3235,
 3236,
 3545,
 3546,
 3547,
 4064,
 4065,
 4066,
 4077,
 4078,
 4079,
 4204,
 4205,
 4206,
 4410,
 4411,
 4412,
 5777,
 5778,
 5779,
 6029,
 6030,
 6031,
 6957,
 6958,
 6959,
 7050,
 7051,
 7052,
 7662,
 7663,
 7664,
 7693,
 7694,
 7695,
 7699,
 7700,
 7701,
 7930,
 7931,
 7932,
 8538,
 8539,
 8540,
 8696,
 8697,
 8698,
 9734,
 9735,
 9736,
 9769,
 9770,
 9771,
 10218,
 10219,
 10220,
 10421,
 10422,
 10423,
 10475,
 10476,
 10477,
 10774,
 10775,
 10776,
 10871,
 10872,
 10873,
 11016,
 11017,
 11018,
 11112,
 11113,
 11114,
 11832,
 11833,
 11834,
 12131,
 12132,
 12133,
 12394,
 12395,
 12396,
 12419,
 12420,
 12421,
 12424,
 12425,
 12426,
 12454,
 12455,
 12456,
 12662,
 12663,
 12664,
 12761,
 12762,
 12763,
 12771,
 12772,
 12773,
 13129,
 13130,
 13131,
 14677,
 14678,
 14679,
 15100,
 15101,
 15102,
 15111,
 1

In [638]:
for k,v in my_tables.items():
    for col in v.print_df['ens'].columns:
        if 'par' in col:
            v.print_df['ens'].drop(col,inplace=True, axis=1)
    if v.print_df['ens'].loc['weighted avg','fixed_f1']>=0.806:
        print(k,fix_options[k])
        with pd.option_context('display.float_format', '{:0.3f}'.format):
            display(v.print_df['ens'])

fm.tm.dm.sm.one.neigh ['four_miss', 'tree_miss', 'double_miss', 'single_miss']


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.874,0.874,0.921,0.898,0.832,0.854
1,0.550,0.601,0.470,0.561,0.677,0.659
weighted avg,0.793,0.806,0.805,0.811,0.797,0.808


fm.tm.dm.sm.ssa ['four_miss', 'tree_miss', 'double_miss', 'single_miss', 'single_sa']


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.874,0.875,0.921,0.901,0.832,0.853
1,0.550,0.600,0.470,0.556,0.677,0.664
weighted avg,0.793,0.807,0.805,0.812,0.797,0.809


sm.dm.tm.fm.shift ['single_miss', 'double_miss', 'tree_miss', 'four_miss']


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.874,0.874,0.921,0.898,0.832,0.854
1,0.550,0.601,0.470,0.561,0.677,0.659
weighted avg,0.793,0.806,0.805,0.811,0.797,0.808


sm.dm.tm.fm.ssa ['single_miss', 'double_miss', 'tree_miss', 'four_miss', 'single_sa']


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.874,0.875,0.921,0.901,0.832,0.853
1,0.550,0.600,0.470,0.556,0.677,0.664
weighted avg,0.793,0.807,0.805,0.812,0.797,0.809


fm.tm.dm.sm_4n ['four_miss', 'tree_miss', 'double_miss', 'single_miss', 'single_sa']


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.874,0.875,0.921,0.901,0.832,0.853
1,0.550,0.600,0.470,0.556,0.677,0.664
weighted avg,0.793,0.807,0.805,0.812,0.797,0.809


In [604]:
my_tables['sm.dm.tm.fm.shift'].my_fixer.fixed_df

,bert_group,bert_split,bert_predicted,bert_true,bert_proba_0,bert_proba_1,bert_sent_idx,bert_par,crf_group,crf_par,...,crf_proba_0,crf_proba_1,crf_sent_idx,ens_par,ens_predicted,ens_group,ens_split,ens_true,ens_par_y,ens_sent_idx
0,4,0,0,0,0.73576,0.26424,0,0,4,0,...,0.92899,0.07101,0,0,0,4,0,0,0,0
1,4,0,0,0,0.80924,0.19076,1,1,4,1,...,0.94880,0.05120,1,1,0,4,0,0,1,1
2,4,0,0,0,0.83017,0.16983,2,2,4,2,...,0.96377,0.03623,2,2,0,4,0,0,2,2
3,4,0,0,0,0.62789,0.37211,3,3,4,3,...,0.89181,0.10819,3,3,0,4,0,0,3,3
4,4,0,0,0,0.65687,0.34313,4,3,4,3,...,0.76947,0.23053,4,3,0,4,0,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38776,79,9,0,0,0.79531,0.20469,254,171,79,171,...,0.97280,0.02720,254,171,0,79,9,0,171,254
38777,79,9,0,0,0.86084,0.13916,255,172,79,172,...,0.98027,0.01973,255,172,0,79,9,0,172,255
38778,79,9,0,0,0.87557,0.12443,256,172,79,172,...,0.98962,0.01038,256,172,0,79,9,0,172,256
38779,79,9,0,0,0.86531,0.13469,257,172,79,172,...,0.98989,0.01011,257,172,0,79,9,0,172,257


## get boundaries for fixd fm,tm,dm,sm


In [639]:
doc_to_use=4
t='fm.tm.dm.sm_4n'

In [640]:
wd = classes.WindowDiff()
start={}
end={}
start[t], end[t] = wd.get_boundaries_indices(my_tables[t].my_fixer.fixed_df.query('ens_group==@doc_to_use')['ens_predicted'].tolist())
bound_db={}
bound_db[t]=pd.DataFrame()
bound_db[t]['start']=start[t]
bound_db[t]['end']=end[t]
common_utils.save_db(bound_db[t],dir_name,'boundary_{}_{}'.format(doc_to_use,t))

WindowDiff init called
Saving /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/boundary_4_fm.tm.dm.sm_4n.csv, 
index False
float_format %.5f


In [309]:
err_stat_db=pd.DataFrame()
for est_name,est_stat in my_tables_double.my_fixer.stat.items():
    for stat_type,stat_val in est_stat.items():
        if stat_type=='near':
            for near_name,near_val in stat_val.items():
                err_stat_db.loc["{}_{}".format(stat_type,near_name),est_name]=near_val
        else:
            err_stat_db.loc["{}".format(stat_type),est_name]=stat_val
#             if stat_type in ['mid','stand_alone']:
#                 err_stat_db.loc["{}_rate".format(stat_type),est_name]=stat_val*100/estimator['total_fp']
#             if 'total' in err_type:
#                 err_stat_db.loc["{}_rate".format(stat_type),est_name]=stat_val*100/(estimator['total_fp']+estimator['total_fn'])
err_stat_db

,bert,crf,ens
near_tot,346.00000,100.00000,156.00000
near_fp,162.00000,48.00000,57.00000
near_fn,184.00000,52.00000,99.00000
mid,1039.00000,23.00000,281.00000
stand_alone,1428.00000,24.00000,220.00000
double_mid_miss,274.00000,6.00000,69.00000
double_stand_alone,501.00000,5.00000,134.00000
total_fp,5613.00000,3253.00000,2279.00000
total_fn,4546.00000,4955.00000,5280.00000


In [302]:
for k,v in my_tables_double.print_df.items():
    print(k)
    for col in v.columns:
        if 'par' in col:
            v.drop(col,inplace=True, axis=1)
    display(v)

bert


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.82023,0.86650,0.80913,0.86215,0.83516,0.87397
1,0.51576,0.62703,0.54572,0.64691,0.50346,0.62513
weighted avg,0.74459,0.80727,0.74004,0.80535,0.75607,0.81584


crf


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.85913,0.85979,0.88497,0.88517,0.83550,0.83659
1,0.54454,0.54781,0.49971,0.50360,0.60624,0.60861
weighted avg,0.78118,0.78250,0.78850,0.78964,0.77993,0.78133


ens


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.87389,0.88203,0.92133,0.92884,0.83226,0.84088
1,0.54962,0.58146,0.46967,0.49906,0.67712,0.71199
weighted avg,0.79277,0.80668,0.80533,0.81818,0.79736,0.81235


In [313]:
for k,v in my_tables.print_df.items():
    print(k)
    for col in v.columns:
        if 'par' in col or 'sent' in col:
            v.drop(col,inplace=True, axis=1)
    with pd.option_context('display.float_format', '{:0.3f}'.format):
        display(v)

bert


,fixed_f1,fixed_recall,fixed_prec
0,0.864,0.856,0.875
1,0.625,0.652,0.617
weighted avg,0.805,0.802,0.814


crf


,fixed_f1,fixed_recall,fixed_prec
0,0.860,0.885,0.837
1,0.548,0.503,0.609
weighted avg,0.782,0.790,0.781


ens


,fixed_f1,fixed_recall,fixed_prec
0,0.882,0.928,0.842
1,0.583,0.502,0.710
weighted avg,0.807,0.818,0.812


In [323]:
doc_to_use=4

In [324]:
imp.reload(classes)
wd = classes.WindowDiff()
start={}
end={}
start['mer'], end['mer'] = wd.get_boundaries_indices(my_tables.my_fixer.fixed_df.query('ens_group==@doc_to_use')['ens_predicted'].tolist())

WindowDiff init called


In [327]:
t='mer'
bound_db={}
bound_db[t]=pd.DataFrame()
bound_db[t]['start']=start[t]
bound_db[t]['end']=end[t]
common_utils.save_db(bound_db[t],dir_name,'boundary_{}_{}'.format(doc_to_use,t))

Saving /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/boundary_4_mer.csv, 
index False
float_format %.5f


In [328]:
scores={}
scores_dict={}
scores[t], scores_dict[t], _, _ = model_utils.get_report_by_unit(my_tables.my_fixer.fixed_df.query('ens_group==@doc_to_use'),'ens',"group")

In [329]:
scores

{'mer': [0.7802652911744257]}

In [312]:
for k,v in my_tables.latex_str.items():
    print(k)
    display(v)

bert


'&sent_f1&fixed_f1&par_f1&sent_recall&fixed_recall&par_recall&sent_prec&fixed_prec&par_prec\\0&0.820&0.864&0.814&0.809&0.856&0.756&0.835&0.875&0.887\\1&0.516&0.625&0.467&0.546&0.652&0.613&0.503&0.617&0.387\\weighted avg&0.745&0.805&0.748&0.740&0.802&0.726&0.756&0.814&0.792\\'

crf


'&sent_f1&fixed_f1&par_f1&sent_recall&fixed_recall&par_recall&sent_prec&fixed_prec&par_prec\\0&0.859&0.860&0.886&0.885&0.885&0.904&0.836&0.837&0.871\\1&0.545&0.548&0.494&0.500&0.503&0.459&0.606&0.609&0.542\\weighted avg&0.781&0.782&0.810&0.789&0.790&0.816&0.780&0.781&0.808\\'

ens


'&sent_f1&fixed_f1&par_f1&sent_recall&fixed_recall&par_recall&sent_prec&fixed_prec&par_prec\\0&0.874&0.882&0.899&0.921&0.928&0.928&0.832&0.842&0.873\\1&0.550&0.583&0.518&0.470&0.502&0.458&0.677&0.710&0.607\\weighted avg&0.793&0.807&0.825&0.805&0.818&0.834&0.797&0.812&0.824\\'

In [305]:
imp.reload(classes)

my_tables_sinle_on_pred=classes.MyScoreSummarizer(all_pred_crf01,False)
my_tables_sinle_on_pred.get_all_scores()

MyScoreSummarizer init called
MyPredFixer init called
WindowDiff init called
bert
near misses to be fixed
 {'tot': 346, 'fp': 162, 'fn': 184}
1890 middle misses to be fixed
1633 stande alone to be fixed
crf
near misses to be fixed
 {'tot': 100, 'fp': 48, 'fn': 52}
0 middle misses to be fixed
0 stande alone to be fixed
ens
near misses to be fixed
 {'tot': 156, 'fp': 57, 'fn': 99}
399 middle misses to be fixed
350 stande alone to be fixed
MyReport init called
MyReport init called
MyReport init called


In [367]:
scores_dict

{0: {'0': {'precision': 0.8285335981463091,
   'recall': 0.9638043896804005,
   'f1-score': 0.891064435742257,
   'support': 2597},
  '1': {'precision': 0.7366946778711485,
   'recall': 0.33674775928297057,
   'f1-score': 0.46221441124780316,
   'support': 781},
  'accuracy': 0.8188277087033747,
  'macro avg': {'precision': 0.7826141380087288,
   'recall': 0.6502760744816856,
   'f1-score': 0.6766394234950301,
   'support': 3378},
  'weighted avg': {'precision': 0.8073002657795535,
   'recall': 0.8188277087033747,
   'f1-score': 0.7919134975746523,
   'support': 3378}},
 1: {'0': {'precision': 0.7917374080362196,
   'recall': 0.9611817244933013,
   'f1-score': 0.8682699767261443,
   'support': 2911},
  '1': {'precision': 0.7322274881516587,
   'recall': 0.29569377990430623,
   'f1-score': 0.42126789366053163,
   'support': 1045},
  'accuracy': 0.7853892821031345,
  'macro avg': {'precision': 0.7619824480939392,
   'recall': 0.6284377521988038,
   'f1-score': 0.6447689351933379,
   'sup

In [306]:
for k,v in my_tables_sinle_on_pred.print_df.items():
    print(k)
    for col in v.columns:
        if 'par' in col:
            v.drop(col,inplace=True, axis=1)
    display(v)

bert


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.82023,0.83640,0.80913,0.82096,0.83516,0.85681
1,0.51576,0.56771,0.54572,0.60928,0.50346,0.54924
weighted avg,0.74459,0.76990,0.74004,0.76504,0.75607,0.78380


crf


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.85913,0.85933,0.88497,0.88480,0.83550,0.83605
1,0.54454,0.54609,0.49971,0.50186,0.60624,0.60690
weighted avg,0.78118,0.78172,0.78850,0.78891,0.77993,0.78050


ens


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.87389,0.87587,0.92133,0.92166,0.83226,0.83565
1,0.54962,0.56061,0.46967,0.48259,0.67712,0.68512
weighted avg,0.79277,0.79693,0.80533,0.80881,0.79736,0.80172


In [307]:
imp.reload(classes)

my_tables_double_on_pred=classes.MyScoreSummarizer(all_pred_crf01,True)
my_tables_double_on_pred.get_all_scores()

MyScoreSummarizer init called
MyPredFixer init called
WindowDiff init called
bert
near misses to be fixed
 {'tot': 346, 'fp': 162, 'fn': 184}
1890 middle misses to be fixed
1633 stande alone to be fixed
501 double stande alone to be fixed
274 double middle misses to be fixed
crf
near misses to be fixed
 {'tot': 100, 'fp': 48, 'fn': 52}
0 middle misses to be fixed
0 stande alone to be fixed
5 double stande alone to be fixed
6 double middle misses to be fixed
ens
near misses to be fixed
 {'tot': 156, 'fp': 57, 'fn': 99}
399 middle misses to be fixed
350 stande alone to be fixed
134 double stande alone to be fixed
69 double middle misses to be fixed
MyReport init called
MyReport init called
MyReport init called


In [308]:
for k,v in my_tables_double_on_pred.print_df.items():
    print(k)
    for col in v.columns:
        if 'par' in col:
            v.drop(col,inplace=True, axis=1)
    display(v)

bert


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.82023,0.83931,0.80913,0.82712,0.83516,0.85616
1,0.51576,0.56875,0.54572,0.60400,0.50346,0.55610
weighted avg,0.74459,0.77229,0.74004,0.76829,0.75607,0.78508


crf


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.85913,0.85933,0.88497,0.88479,0.83550,0.83606
1,0.54454,0.54622,0.49971,0.50200,0.60624,0.60706
weighted avg,0.78118,0.78174,0.78850,0.78893,0.77993,0.78054


ens


,sent_f1,fixed_f1,sent_recall,fixed_recall,sent_prec,fixed_prec
0,0.87389,0.87609,0.92133,0.92298,0.83226,0.83501
1,0.54962,0.55927,0.46967,0.47978,0.67712,0.68745
weighted avg,0.79277,0.79668,0.80533,0.80891,0.79736,0.80190


### Try to train ensemble on merged predictions

In [333]:
cv_splits_loaded=common_utils.load_json(dir_name,cv_split_name)
cv_splits_loaded

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/cv_splits_10.json


{'0': {'test': [4, 14, 15, 18, 29, 32, 36, 70],
  'train': [1,
   2,
   3,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   16,
   17,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   30,
   31,
   33,
   34,
   35,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80]},
 '1': {'test': [5, 6, 13, 34, 37, 62, 73, 80],
  'train': [1,
   2,
   3,
   4,
   7,
   8,
   9,
   10,
   11,
   12,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   35,
   36,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   6

In [336]:
fixed_pred=my_tables_sinle_on_pred.my_fixer.fixed_df.copy()

In [337]:
from sklearn.svm import SVC
imp.reload(model_utils)
ens_pred=model_utils.prepared_cross_validate_ensemble(SVC(random_state=0),fixed_pred,cv_splits_loaded,False)

0 split started...
1 split started...
2 split started...
3 split started...
4 split started...
5 split started...
6 split started...
7 split started...
8 split started...
9 split started...


In [338]:
scores, scores_dict, _, _ = model_utils.get_report_by_unit(ens_pred,'ens',"split")

In [340]:
np.mean(scores)

0.7748702825183011

In [342]:
ens_pred_plane=model_utils.prepared_cross_validate_ensemble(SVC(random_state=0),my_tables_sinle_on_pred.my_fixer.pred_df,cv_splits_loaded,False)

0 split started...
1 split started...
2 split started...
3 split started...
4 split started...
5 split started...
6 split started...
7 split started...
8 split started...
9 split started...


In [343]:
scores_plane, scores_dict_plane, _, _ = model_utils.get_report_by_unit(ens_pred_plane,'ens',"split")
np.mean(scores_plane)

0.7617807647672052

### Add plot of fixed ensemble

In [345]:
fixed_f1_score=common_utils.load_db(dir_name,'bert.cfg.svc.weighted_f1.fixed',True)

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/bert.cfg.svc.weighted_f1.fixed.csv,  index True


In [347]:
pure_f1_score=common_utils.load_db(dir_name,'bert.cfg.svc.weighted_f1.pure',True)

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/bert.cfg.svc.weighted_f1.pure.csv,  index True


In [350]:
f1_score=common_utils.load_db(dir_name,'bert.cfg.svc.weighted_f1',True)

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/bert.cfg.svc.weighted_f1.csv,  index True


In [348]:
fixed_f1_score

,Unnamed: 0,bert,crf,ens
0,0,0.83515,0.81135,0.83368
1,1,0.80885,0.75450,0.78396
2,2,0.81342,0.77111,0.78792
3,3,0.77587,0.74848,0.78180
4,4,0.79152,0.78009,0.78384
5,5,0.81002,0.76068,0.78793
6,6,0.84801,0.83285,0.85487
7,7,0.77328,0.81058,0.82617
8,8,0.80453,0.82589,0.84774
9,9,0.78907,0.72938,0.78169


In [349]:
pure_f1_score

,Unnamed: 0,bert,crf,ens
0,0,0.78029,0.80963,0.81835
1,1,0.73063,0.75374,0.77188
2,2,0.75184,0.76873,0.77385
3,3,0.71842,0.74690,0.76610
4,4,0.72983,0.77839,0.77550
5,5,0.75766,0.75900,0.77321
6,6,0.79431,0.83151,0.83864
7,7,0.72026,0.80980,0.81446
8,8,0.74909,0.82563,0.83403
9,9,0.71354,0.72851,0.76171


In [351]:
f1_score

,Unnamed: 0,base,bert,crf,ens
0,0,0.75156,0.83515,0.81135,0.83368
1,1,0.71553,0.80885,0.75450,0.78396
2,2,0.72088,0.81342,0.77111,0.78792
3,3,0.71118,0.77587,0.74848,0.78180
4,4,0.71024,0.79152,0.78009,0.78384
5,5,0.71019,0.81002,0.76068,0.78793
6,6,0.79182,0.84801,0.83285,0.85487
7,7,0.76333,0.77328,0.81058,0.82617
8,8,0.78739,0.80453,0.82589,0.84774
9,9,0.65185,0.78907,0.72938,0.78169


In [352]:
full_scores=pure_f1_score.merge(f1_score[['base']],left_index=True,right_index=True,validate='one_to_one')

In [354]:
full_scores=full_scores.merge(fixed_f1_score[['ens']],left_index=True,right_index=True,validate='one_to_one')
full_scores

,Unnamed: 0,bert,crf,ens_x,base,ens_y
0,0,0.78029,0.80963,0.81835,0.75156,0.83368
1,1,0.73063,0.75374,0.77188,0.71553,0.78396
2,2,0.75184,0.76873,0.77385,0.72088,0.78792
3,3,0.71842,0.74690,0.76610,0.71118,0.78180
4,4,0.72983,0.77839,0.77550,0.71024,0.78384
5,5,0.75766,0.75900,0.77321,0.71019,0.78793
6,6,0.79431,0.83151,0.83864,0.79182,0.85487
7,7,0.72026,0.80980,0.81446,0.76333,0.82617
8,8,0.74909,0.82563,0.83403,0.78739,0.84774
9,9,0.71354,0.72851,0.76171,0.65185,0.78169


In [355]:
full_scores.rename({'ens_x':'ens','ens_y':'fixed'},axis=1,inplace=True)
full_scores

,Unnamed: 0,bert,crf,ens,base,fixed
0,0,0.78029,0.80963,0.81835,0.75156,0.83368
1,1,0.73063,0.75374,0.77188,0.71553,0.78396
2,2,0.75184,0.76873,0.77385,0.72088,0.78792
3,3,0.71842,0.74690,0.76610,0.71118,0.78180
4,4,0.72983,0.77839,0.77550,0.71024,0.78384
5,5,0.75766,0.75900,0.77321,0.71019,0.78793
6,6,0.79431,0.83151,0.83864,0.79182,0.85487
7,7,0.72026,0.80980,0.81446,0.76333,0.82617
8,8,0.74909,0.82563,0.83403,0.78739,0.84774
9,9,0.71354,0.72851,0.76171,0.65185,0.78169


In [357]:
full_scores.loc['mean'] = full_scores.mean()

In [361]:
common_utils.save_db(full_scores,dir_name,'bert.cfg.svc.weighted_f1.pure.and.fixed',False)

Saving /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/bert.cfg.svc.weighted_f1.pure.and.fixed.csv, 
index False
float_format %.5f


In [360]:
full_scores

,Unnamed: 0,bert,crf,ens,base,fixed
0,0.00000,0.78029,0.80963,0.81835,0.75156,0.83368
1,1.00000,0.73063,0.75374,0.77188,0.71553,0.78396
2,2.00000,0.75184,0.76873,0.77385,0.72088,0.78792
3,3.00000,0.71842,0.74690,0.76610,0.71118,0.78180
4,4.00000,0.72983,0.77839,0.77550,0.71024,0.78384
5,5.00000,0.75766,0.75900,0.77321,0.71019,0.78793
6,6.00000,0.79431,0.83151,0.83864,0.79182,0.85487
7,7.00000,0.72026,0.80980,0.81446,0.76333,0.82617
8,8.00000,0.74909,0.82563,0.83403,0.78739,0.84774
9,9.00000,0.71354,0.72851,0.76171,0.65185,0.78169


In [363]:
full_scores.loc['mean'].transpose()

Unnamed: 0   4.50000
bert         0.74459
crf          0.78118
ens          0.79277
base         0.73140
fixed        0.80696
Name: mean, dtype: float64

In [364]:
per_label=common_utils.load_db(dir_name,"bert.cfg.svc.per.fold_f1.pure")

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/bert.cfg.svc.per.fold_f1.pure.csv,  index False


In [365]:
per_label

,Unnamed: 0,bert_1,bert_0,crf_1,crf_0,ens_1,ens_0
0,1,0.52347,0.85753,0.55279,0.88687,0.55172,0.89853
1,2,0.52371,0.80492,0.46722,0.85659,0.49597,0.87093
2,3,0.53370,0.83174,0.54064,0.85227,0.51705,0.86791
3,4,0.53235,0.78910,0.53751,0.82643,0.55607,0.84588
4,5,0.54094,0.81768,0.64000,0.84276,0.59779,0.85815
5,6,0.53488,0.84501,0.51271,0.85557,0.52308,0.87129
6,7,0.47746,0.86698,0.53502,0.89951,0.54207,0.90666
7,8,0.46407,0.79090,0.54009,0.88417,0.54881,0.88770
8,9,0.42435,0.81908,0.49487,0.89691,0.51558,0.90266
9,10,0.60267,0.77933,0.62458,0.79018,0.64804,0.82917


In [386]:
base_per_label=common_utils.load_db(dir_name,"base.per.label")
base_per_label

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/base.per.label.csv,  index False


,0.0_ngr,1.0_ngr,weighted avg_ngr
0,0.86936,0.35983,0.75156
1,0.84815,0.34608,0.71553
2,0.84931,0.37026,0.72088
3,0.83789,0.37762,0.71118
4,0.83739,0.40243,0.71024
5,0.84420,0.36842,0.71019
6,0.88865,0.36963,0.79182
7,0.87301,0.36558,0.76333
8,0.88595,0.33010,0.78739
9,0.78983,0.41932,0.65185


In [397]:
all_par_label=db.merge(base_per_label,left_index=True,right_index=True,validate='one_to_one',how='inner')

In [398]:
all_par_label.loc['mean']=all_par_label.mean()

In [399]:
all_par_label

,0_ens,1_ens,weighted avg_ens,0_crf,1_crf,weighted avg_crf,0_bert,1_bert,weighted avg_bert,0_fixed,1_fixed,weighted avg_fixed,0.0_ngr,1.0_ngr,weighted avg_ngr
0,0.89853,0.55172,0.81835,0.88687,0.55279,0.80963,0.85753,0.52347,0.78029,0.90151,0.56315,0.82328,0.86936,0.35983,0.75156
1,0.87093,0.49597,0.77188,0.85659,0.46722,0.75374,0.80492,0.52371,0.73063,0.87325,0.50619,0.77629,0.84815,0.34608,0.71553
2,0.86791,0.51705,0.77385,0.85227,0.54064,0.76873,0.83174,0.53370,0.75184,0.86974,0.52746,0.77798,0.84931,0.37026,0.72088
3,0.84588,0.55607,0.76610,0.82643,0.53751,0.74690,0.78910,0.53235,0.71842,0.84807,0.56865,0.77115,0.83789,0.37762,0.71118
4,0.85815,0.59779,0.77550,0.84276,0.64000,0.77839,0.81768,0.54094,0.72983,0.85750,0.59538,0.77428,0.83739,0.40243,0.71024
5,0.87129,0.52308,0.77321,0.85557,0.51271,0.75900,0.84501,0.53488,0.75766,0.87235,0.52733,0.77517,0.84420,0.36842,0.71019
6,0.90666,0.54207,0.83864,0.89951,0.53502,0.83151,0.86698,0.47746,0.79431,0.91051,0.56969,0.84692,0.88865,0.36963,0.79182
7,0.88770,0.54881,0.81446,0.88417,0.54009,0.80980,0.79090,0.46407,0.72026,0.88856,0.55588,0.81666,0.87301,0.36558,0.76333
8,0.90266,0.51558,0.83403,0.89691,0.49487,0.82563,0.81908,0.42435,0.74909,0.90417,0.52862,0.83758,0.88595,0.33010,0.78739
9,0.82917,0.64804,0.76171,0.79018,0.62458,0.72851,0.77933,0.60267,0.71354,0.83304,0.66374,0.76999,0.78983,0.41932,0.65185


In [421]:
all_par_label.loc['mean',all_par_label.columns.str.contains('ens')]

0_ens              0.87389
1_ens              0.54962
weighted avg_ens   0.79277
Name: mean, dtype: float64

In [406]:
all_par_label.rename({'0.0_ngr':'0_ngr','1.0_ngr':'1_ngr'},axis=1,inplace=True)

In [407]:
common_utils.save_db(all_par_label,dir_name,"all.per_label.mean")

Saving /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/all.per_label.mean.csv, 
index False
float_format %.5f


In [408]:
all_par_label.loc['mean'].transpose()

0_ens                0.87389
1_ens                0.54962
weighted avg_ens     0.79277
0_crf                0.85913
1_crf                0.54454
weighted avg_crf     0.78118
0_bert               0.82023
1_bert               0.51576
weighted avg_bert    0.74459
0_fixed              0.87587
1_fixed              0.56061
weighted avg_fixed   0.79693
0_ngr                0.85237
1_ngr                0.37093
weighted avg_ngr     0.73140
Name: mean, dtype: float64

In [410]:
db_to_plot=pd.DataFrame()
for c in all_par_label.columns:
    splited=c.split('_')
    l=splited[0]
    t=splited[-1]
    print('l:{}, t:{}'.format(l,t))
    db_to_plot.loc[t,l]=all_par_label.loc['mean',c]
db_to_plot
        

l:0, t:ens
l:1, t:ens
l:weighted avg, t:ens
l:0, t:crf
l:1, t:crf
l:weighted avg, t:crf
l:0, t:bert
l:1, t:bert
l:weighted avg, t:bert
l:0, t:fixed
l:1, t:fixed
l:weighted avg, t:fixed
l:0, t:ngr
l:1, t:ngr
l:weighted avg, t:ngr


,0,1,weighted avg
ens,0.87389,0.54962,0.79277
crf,0.85913,0.54454,0.78118
bert,0.82023,0.51576,0.74459
fixed,0.87587,0.56061,0.79693
ngr,0.85237,0.37093,0.73140


In [411]:
common_utils.save_db(db_to_plot,dir_name,"f1.per_label.to_plot")

Saving /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/f1.per_label.to_plot.csv, 
index False
float_format %.5f


In [418]:
trans=db_to_plot.transpose()

In [419]:
trans

,ens,crf,bert,fixed,ngr
0,0.87389,0.85913,0.82023,0.87587,0.85237
1,0.54962,0.54454,0.51576,0.56061,0.37093
weighted avg,0.79277,0.78118,0.74459,0.79693,0.73140


In [420]:
common_utils.save_db(trans,dir_name,"f1.per_label.to_plot",True)

Saving /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/f1.per_label.to_plot.csv, 
index True
float_format %.5f


### Get plain ensemble values for f1-score, prec and rec

In [422]:
ens_pred=common_utils.load_db(dir_name,'ens.SVC.pred.new.76.sent_idx')

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/ens.SVC.pred.new.76.sent_idx.csv,  index False


In [423]:
sc,dic,_,_=model_utils.get_report_by_unit(ens_pred,'ens',"split")

In [424]:
dic

{0: {'0': {'precision': 0.8499313186813187,
   'recall': 0.9530227185213708,
   'f1-score': 0.8985296787075693,
   'support': 2597},
  '1': {'precision': 0.7381974248927039,
   'recall': 0.44046094750320103,
   'f1-score': 0.5517241379310345,
   'support': 781},
  'accuracy': 0.8345174659561871,
  'macro avg': {'precision': 0.7940643717870113,
   'recall': 0.6967418330122859,
   'f1-score': 0.7251269083193019,
   'support': 3378},
  'weighted avg': {'precision': 0.8240982307449931,
   'recall': 0.8345174659561871,
   'f1-score': 0.818347580618027,
   'support': 3378}},
 1: {'0': {'precision': 0.8096221959858324,
   'recall': 0.9422878735829612,
   'f1-score': 0.8709318939514209,
   'support': 2911},
  '1': {'precision': 0.704225352112676,
   'recall': 0.3827751196172249,
   'f1-score': 0.4959702417854928,
   'support': 1045},
  'accuracy': 0.7944893832153691,
  'macro avg': {'precision': 0.7569237740492543,
   'recall': 0.662531496600093,
   'f1-score': 0.6834510678684569,
   'support'

In [430]:
db_ens=pd.DataFrame()
for fold,v in dic.items():
    for l in ['0','1','weighted avg']:
        for sc in ['f1-score','recall','precision']:
            db_ens.loc[fold,'{}_{}'.format(l,sc)]=v[l][sc]
db_ens.loc['mean']=db_ens.mean()

In [432]:
with pd.option_context('display.float_format', '{:0.3f}'.format):
    display(db_ens)

,0_f1-score,0_recall,0_precision,1_f1-score,1_recall,1_precision,weighted avg_f1-score,weighted avg_recall,weighted avg_precision
0,0.899,0.953,0.850,0.552,0.440,0.738,0.818,0.835,0.824
1,0.871,0.942,0.810,0.496,0.383,0.704,0.772,0.794,0.782
2,0.868,0.931,0.813,0.517,0.414,0.688,0.774,0.793,0.779
3,0.846,0.866,0.826,0.556,0.521,0.597,0.766,0.771,0.763
4,0.858,0.930,0.797,0.598,0.491,0.764,0.775,0.790,0.787
5,0.871,0.955,0.801,0.523,0.395,0.775,0.773,0.797,0.794
6,0.907,0.926,0.888,0.542,0.492,0.604,0.839,0.845,0.835
7,0.888,0.907,0.869,0.549,0.506,0.599,0.814,0.820,0.811
8,0.903,0.914,0.892,0.516,0.487,0.548,0.834,0.838,0.831
9,0.829,0.889,0.777,0.648,0.569,0.753,0.762,0.770,0.768


### get table with results of all ensemble models

In [433]:
ens_list=glob.glob(os.path.join(os.path.join(os.getcwd(), "dataframes","reparse","ens*pred.new.76.csv")))

In [434]:
ens_list

['/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/ens.SGDClassifier.pred.new.76.csv',
 '/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/ens.SVC.pred.new.76.csv',
 '/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/ens.RidgeClassifier.pred.new.76.csv',
 '/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/ens.PassiveAggressiveClassifier.pred.new.76.csv',
 '/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/ens.RidgeClassifierCV.pred.new.76.csv',
 '/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/ens.DecisionTreeClassifier.pred.new.76.csv',
 '/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/ens.LogisticRegressionCV.pred.new.76.csv',
 '/scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/ens.Perceptron.pre

In [449]:
all_ens=pd.DataFrame()
from sklearn.metrics import f1_score
for l in ens_list:
    pred_db=pd.read_csv(l)
#     score=f1_score(pred_db['ens_true'].tolist(),pred_db['ens_predicted'].tolist())
    score,_,_,_=model_utils.get_report_by_unit(pred_db,'ens','split')
    all_ens.loc[l.split('.')[1],'f1-score']=np.mean(score)
    
#     score

/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zsofya/jupyter_git/jup-nb-generic/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [455]:
all_ens.sort_values(by='f1-score',ascending=False,inplace=True)

In [457]:
common_utils.save_db(all_ens,dir_name,"all.ens.f1",True,float_format='%.3f')

Saving /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/all.ens.f1.csv, 
index True
float_format %.3f
